In [9]:
#import findspark
#findspark.init('/home/max/Downloads/spark-2.4.0-bin-hadoop2.7')
#from pyspark.sql import SparkSession



In [10]:
#spark= SparkSession.builder.getOrCreate()

In [1]:

import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import csv

In [5]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)

In [2]:
# import code counts, and convert to dict with probabilities
code_counts = pd.read_csv('code_counts.csv')
code_counts.columns = ['code', 'count']
code_counts['prop'] = code_counts['count']/code_counts['count'].sum()
#code_counts.head()
code_counts = dict(zip(code_counts['code'], code_counts['prop']))


In [4]:
# get random code based on the weight (probability) of each code
def get_random_code(dct):
    rand_val = random.random()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'
    
get_random_code(code_counts)

'H04'

In [182]:
# from collections import Counter
# acc = []
# for i in range(10000):
#     acc.append(weighted_random_by_dct(code_counts))
# Counter(xx).most_common()

In [6]:
start_value = 5
start_time_shift =20
random_walk_weight = 0.01
random_walk_variance = 1
seasonality_1_freq = 50 #lower is shorter
seasonality_1_weight = 5
seasonality_2_freq = 7 #lower is shorter
seasonality_2_weight = 2

sample_size = 1000

trend = 0.02
exponent = 1
codes = list('abcdefghijklmnopqrstuvwxyz')
n_samples = 10

terms = []
with open('wordlist.csv', 'r') as f:
    reader = csv.reader(f)
    terms = [word[0] for word in list(reader)]
    random.shuffle(terms)

# Generates time series
def generate_ts():
    sample = []
    random_walk = np.random.normal(0,random_walk_variance)
    y = start_value

    for time in np.arange(sample_size):

        #Assume no start at y = 0
        time = time + start_time_shift

        #Add seasonality 1
        y = (np.sin(time/seasonality_1_freq))*seasonality_1_weight
        

        #Add seasonality 2
        y = y + np.sin(time/seasonality_2_freq)*seasonality_2_weight

        #Add trend
        y = y + (trend * (time ** exponent))

        # Random walk component
        random_walk = random_walk + np.random.normal(0,random_walk_variance)
        y = y + random_walk * random_walk_weight
        

        sample.append(max(0, y))
    return sample

#Plot some time series
def plot_ts(n):
    for _ in range(n):
        plt.plot(generate_ts())

#Creates a dummy dataset from n time series for 'codes'

#For profiling by line:
#kernprof -l generate_data.py
#python3 -m line_profiler generate_data.py.lprof
#uncomment following line:
#@profile 
def create_dummy_dataset():
    df = []
    sample_range = 0
    samples = generate_ts()
    for index, number in enumerate(samples): #Loop over a sample
        for n in range(int(round(number))): #Create n entries
            date = datetime.datetime(1950, 1, 1, 0, 0) + timedelta(days=index)
            current_sample = index+n
            sample = terms[current_sample:current_sample+100]
            text = " ".join(sample)
            df.append([date, get_random_code(code_counts), text])
    pandas_df = pd.DataFrame(df, columns=['date', 'code', 'text'])
    return pandas_df


dummy_dataset = create_dummy_dataset()
#dummy_dataset

# plot_ts(10)
# plt.show()

In [7]:
dummy_dataset.head()

,date,code,text
0,1950-01-01,Y10,protective then talented initiatives ah nonprofit boats trials commerce resource thin las gg ob governments mile unified gamecube issues tribune selection titanium nomination knowledge arrivals insects request consoles busy patrick reservation agreement aj disabled works nine whereas came bee heel love roots clocks orleans liked induced ranks receiving beatles magazine ellen purposes crest preceding trader paraguay busty italic browsers quilt towards premiere nipples textiles always distributor contracts quoted zero faster losing generators applicants indication zimbabwe acting estimates auctions consider op continent hit genesis citizenship connector ol shut bond fell edgar motivation holmes nasty andrea robust ranking rome acdbentity destiny occurrence
1,1950-01-01,F25,then talented initiatives ah nonprofit boats trials commerce resource thin las gg ob governments mile unified gamecube issues tribune selection titanium nomination knowledge arrivals insects request consoles busy patrick reservation agreement aj disabled works nine whereas came bee heel love roots clocks orleans liked induced ranks receiving beatles magazine ellen purposes crest preceding trader paraguay busty italic browsers quilt towards premiere nipples textiles always distributor contracts quoted zero faster losing generators applicants indication zimbabwe acting estimates auctions consider op continent hit genesis citizenship connector ol shut bond fell edgar motivation holmes nasty andrea robust ranking rome acdbentity destiny occurrence swimming
2,1950-01-01,A61,talented initiatives ah nonprofit boats trials commerce resource thin las gg ob governments mile unified gamecube issues tribune selection titanium nomination knowledge arrivals insects request consoles busy patrick reservation agreement aj disabled works nine whereas came bee heel love roots clocks orleans liked induced ranks receiving beatles magazine ellen purposes crest preceding trader paraguay busty italic browsers quilt towards premiere nipples textiles always distributor contracts quoted zero faster losing generators applicants indication zimbabwe acting estimates auctions consider op continent hit genesis citizenship connector ol shut bond fell edgar motivation holmes nasty andrea robust ranking rome acdbentity destiny occurrence swimming schemes
3,1950-01-02,H01,then talented initiatives ah nonprofit boats trials commerce resource thin las gg ob governments mile unified gamecube issues tribune selection titanium nomination knowledge arrivals insects request consoles busy patrick reservation agreement aj disabled works nine whereas came bee heel love roots clocks orleans liked induced ranks receiving beatles magazine ellen purposes crest preceding trader paraguay busty italic browsers quilt towards premiere nipples textiles always distributor contracts quoted zero faster losing generators applicants indication zimbabwe acting estimates auctions consider op continent hit genesis citizenship connector ol shut bond fell edgar motivation holmes nasty andrea robust ranking rome acdbentity destiny occurrence swimming
4,1950-01-02,H02,talented initiatives ah nonprofit boats trials commerce resource thin las gg ob governments mile unified gamecube issues tribune selection titanium nomination knowledge arrivals insects request consoles busy patrick reservation agreement aj disabled works nine whereas came bee heel love roots clocks orleans liked induced ranks receiving beatles magazine ellen purposes crest preceding trader paraguay busty italic browsers quilt towards premiere nipples textiles always distributor contracts quoted zero faster losing generators applicants indication zimbabwe acting estimates auctions consider op continent hit genesis citizenship connector ol shut bond fell edgar motivation holmes nasty andrea robust ranking rome acdbentity destiny occurrence swimming schemes
